In [2]:
from requests import get
from bs4 import BeautifulSoup

In [4]:
#creating loop to create urls for 50 movies at a time

#initializing iterator variable (pages increment in form of 'start=1', then 'start=51', then 'start=101'......)
z = 1

#creating empty list for urls to be stored in
urls = []

#while the number of movies is below 4002 (arbitrary #)
while z < 4002:
    
    #generating url using format to insert the incrementing variable on each loop
    url = 'https://www.imdb.com/search/title/?title_type=feature&moviemeter=,4000&sort=user_rating,desc&start={}&ref_=adv_nxt'.format(z)
    #try and use this url at some point
    #https://www.imdb.com/search/title/?title_type=feature&release_date=1950-01-01,2020-06-30&user_rating=4.0,&start=1&ref_=adv_nxt
    
    #appending the url to the list of urls
    urls.append(url)
    
    #incrementing iterator variable so it can go to the next page (there are 50 movies listed per page)
    z += 50

In [5]:
#for loop to get all the html objects of each array into a soup object

#creating empty soup item to store the generated html files in
html_soups = BeautifulSoup()

#iterating through the url pages
for url in urls:
    
    #creating response to use in BeautifulSoup function
    response = get(url)
    
    #using the BeautifulSoup function on each response, appending the resulting soup object into the empty 'html_soups'
    html_soups.append(BeautifulSoup(response.text, 'html.parser'))

In [6]:
#creating containers for each movie, seperating at 'lister-item mode-advanced' on html doc
m_containers = html_soups.find_all('div', class_='lister-item mode-advanced')

#printing to make sure the results are what we expect; if you go to the URL, there are only 2758 items on the list created, all are here
print(type(m_containers))
print(len(m_containers))

<class 'bs4.element.ResultSet'>
2747


In [181]:
#creating empty list to store dictionaries of each movie info
movies = []

#for loop to parse through each movie in m_containers and get the info
for container in m_containers:
    
    #finding title of movie through its location in html doc
    title = container.h3.a.text
    
    #finding link for each movies imdb page
    #for loop to go through the container and find the correct link
    for a in container.h3.find_all('a', href=True):
        m_link = ('imdb.com'+a['href'])


    
    #finding year of movie through its location in html doc
    #f container.h3.find('span', class_='lister-item-year').text != '(2021)':
    year = container.h3.find('span', class_='lister-item-year').text.replace('(', '').replace(')', '')


    
    #if function created to deal with any movies that dont have ratings
   
    #if there is a rating found in the html
    if container.strong is not None:
        
        #imdb rating stored in i_rate
        i_rate = float(container.strong.text)
        
    #if there is NO imdb rating this means the movie isnt out yet
    else:
        
        #set i_rate to 0
        i_rate = 0


    #finding mpaa rating (G/PG/R etc.) in html
    rating = container.p.span.text

    #finding director in html
    if container.find_all('p')[2].a is not None:
        direct = container.find_all('p')[2].a.text
    else:
        direct = 'NA'

    #finding summary of movie in html
    summary = container.find_all('p', class_='text-muted')[1].text.strip('\n   ')
    
   
    
    #if the movie has credits, and the length of credits is over 0
    #if container.find_all('p')[2].find_all('a') is not None and len(container.find_all('p')[2].find_all('a')) > 0:
    
    #setting credits to text from the html location
    credits = container.find_all('p')[2].text

    #if function used to determine if there is more than one star
    #creating a list y containing element y[0], a list of the director(s), and y[1], a list of the star(s)
    if 'Stars:' in credits:      
        y = credits.split('Stars:')

    elif 'Star:' in starlist:
        y = credits.split('Star:')

    #stripping the list made from above, and splitting it at ', \n' which appears between each star
    #resulting in completed list of stars
    stars = y[1].strip().split(', \n')

    #creating empty lists for directors
    directors=[]
    
    #stripping and splitting the list of directors at ',\n' between each director
    dirs = y[0].strip().split(', \n')
    
    #for loop to strip text of dirs
    for i in dirs:       
        i = i.replace('\n|','')
        if 'Director:' in i:
            i = i.replace('Director:\n','')

        elif 'Directors:' in i:
            i = i.replace('Directors:\n','')
        
        #appending the new text to directors list
        directors.append(i)
    
    #if there is no imdb rating and it was previously set to zero, the movie is not added to the dictionary
    if i_rate != 0:
        
        #appending the values for movie to dictionaries, creating new 'movie' class for each movie
        movies.append({'movie':{'Title':title,'Movie Link':m_link, 'Year':year, 'IMDB Rating':i_rate, "Summary": summary, "MPAA Rating":rating, "Director(s)":directors, "Stars":stars}})
movies[:5]

[{'movie': {'Title': 'The Shawshank Redemption',
   'Movie Link': 'imdb.com/title/tt0111161/',
   'Year': '1994',
   'IMDB Rating': 9.3,
   'Summary': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.',
   'MPAA Rating': 'R',
   'Director(s)': ['Frank Darabont'],
   'Stars': ['Tim Robbins',
    'Morgan Freeman',
    'Bob Gunton',
    'William Sadler']}},
 {'movie': {'Title': 'The Godfather',
   'Movie Link': 'imdb.com/title/tt0068646/',
   'Year': '1972',
   'IMDB Rating': 9.2,
   'Summary': 'The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.',
   'MPAA Rating': 'R',
   'Director(s)': ['Francis Ford Coppola'],
   'Stars': ['Marlon Brando', 'Al Pacino', 'James Caan', 'Diane Keaton']}},
 {'movie': {'Title': 'The Dark Knight',
   'Movie Link': 'imdb.com/title/tt0468569/',
   'Year': '2008',
   'IMDB Rating': 9.0,
   'Summary': 'When the menace known as the

In [175]:
from pymongo import MongoClient

In [177]:
client = MongoClient()

client = MongoClient('localhost', 27017)

db = client.movie_database

collection = db.movies_db

In [178]:
for movie in movies:   
    collection.insert_one(movie)

In [179]:
for article in collection.find().limit(5):
    print(article)

{'_id': ObjectId('5efa3e60d15ed45e4a455a95'), 'Title': 'The Shawshank Redemption', 'Movie Link': 'imdb.com/title/tt0111161/', 'Year': '1994', 'IMDB Rating': 9.3, 'Summary': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'MPAA Rating': 'R', 'Director': 'Frank Darabont'}
{'_id': ObjectId('5efa3e61d15ed45e4a455a96'), 'Title': 'The Godfather', 'Movie Link': 'imdb.com/title/tt0068646/', 'Year': '1972', 'IMDB Rating': 9.2, 'Summary': 'The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.', 'MPAA Rating': 'R', 'Director': 'Francis Ford Coppola'}
{'_id': ObjectId('5efa3e61d15ed45e4a455a97'), 'Title': 'The Dark Knight', 'Movie Link': 'imdb.com/title/tt0468569/', 'Year': '2008', 'IMDB Rating': 9.0, 'Summary': 'When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical test

In [623]:
len(movies)

2654

In [641]:
import pandas as pd
from pandas.io.json import json_normalize
df = json_normalize(movies)
df.head()

,movie.Director,movie.IMDB Rating,movie.MPAA Rating,movie.Movie Link,movie.Summary,movie.Title,movie.Year
0,Frank Darabont,9.3,R,imdb.com/title/tt0111161/,Two imprisoned men bond over a number of years...,The Shawshank Redemption,1994
1,Francis Ford Coppola,9.2,R,imdb.com/title/tt0068646/,The aging patriarch of an organized crime dyna...,The Godfather,1972
2,Christopher Nolan,9.0,PG-13,imdb.com/title/tt0468569/,When the menace known as the Joker wreaks havo...,The Dark Knight,2008
3,Francis Ford Coppola,9.0,R,imdb.com/title/tt0071562/,The early life and career of Vito Corleone in ...,The Godfather: Part II,1974
4,Peter Jackson,8.9,PG-13,imdb.com/title/tt0167260/,Gandalf and Aragorn lead the World of Men agai...,The Lord of the Rings: The Return of the King,2003


In [ ]:
#checking if all le
print(len(titles))
print(len(summs))
print(len(years))
print(len(imdb_ratings))
print(len(mpaa_rates))

To do:
1. Fix the stars section, how to add them to dict

2. find a way to get the documents transferred to mongodb

3. Begin process of creating functionality - research how to do the things we want to do

4. Optimize speed - could the m_containers loop be inside of the "for url in urls" loop? that may make the operation faster

In [ ]:
titles_years = [{"Title": t, "Year": s} for t, s in zip(titles, years)]
titles_years[:11]

In [ ]:
import json
print(json.dumps(movies))